# Information Retrieval 1#
## Assignment 2: Retrieval models [100 points] ##

In this assignment you will get familiar with basic and advanced information retrieval concepts. You will implement different information retrieval ranking models and evaluate their performance.

We provide you with a Indri index. To query the index, you'll use a Python package ([pyndri](https://github.com/cvangysel/pyndri)) that allows easy access to the underlying document statistics.

For evaluation you'll use the [TREC Eval](https://github.com/usnistgov/trec_eval) utility, provided by the National Institute of Standards and Technology of the United States. TREC Eval is the de facto standard way to compute Information Retrieval measures and is frequently referenced in scientific papers.

This is a **groups-of-three assignment**, the deadline is **Wednesday, January 31st**. Code quality, informative comments and convincing analysis of the results will be considered when grading. Submission should be done through blackboard, questions can be asked on the course [Piazza](piazza.com/university_of_amsterdam/spring2018/52041inr6y/home).

### Technicalities (must-read!) ###

The assignment directory is organized as follows:
   * `./assignment.ipynb` (this file): the description of the assignment.
   * `./index/`: the index we prepared for you.
   * `./ap_88_90/`: directory with ground-truth and evaluation sets:
      * `qrel_test`: test query relevance collection (**test set**).
      * `qrel_validation`: validation query relevance collection (**validation set**).
      * `topics_title`: semicolon-separated file with query identifiers and terms.

You will need the following software packages (tested with Python 3.5 inside [Anaconda](https://conda.io/docs/user-guide/install/index.html)):
   * Python 3.5 and Jupyter
   * Indri + Pyndri (Follow the installation instructions [here](https://github.com/nickvosk/pyndri/blob/master/README.md))
   * gensim [link](https://radimrehurek.com/gensim/install.html)
   * TREC Eval [link](https://github.com/usnistgov/trec_eval)

### TREC Eval primer ###
The TREC Eval utility can be downloaded and compiled as follows:

    git clone https://github.com/usnistgov/trec_eval.git
    cd trec_eval
    make

TREC Eval computes evaluation scores given two files: ground-truth information regarding relevant documents, named *query relevance* or *qrel*, and a ranking of documents for a set of queries, referred to as a *run*. The *qrel* will be supplied by us and should not be changed. For every retrieval model (or combinations thereof) you will generate a run of the top-1000 documents for every query. The format of the *run* file is as follows:

    $query_identifier Q0 $document_identifier $rank_of_document_for_query $query_document_similarity $run_identifier
    
where
   * `$query_identifier` is the unique identifier corresponding to a query (usually this follows a sequential numbering).
   * `Q0` is a legacy field that you can ignore.
   * `$document_identifier` corresponds to the unique identifier of a document (e.g., APXXXXXXX where AP denotes the collection and the Xs correspond to a unique numerical identifier).
   * `$rank_of_document_for_query` denotes the rank of the document for the particular query. This field is ignored by TREC Eval and is only maintained for legacy support. The ranks are computed by TREC Eval itself using the `$query_document_similarity` field (see next). However, it remains good practice to correctly compute this field.
   * `$query_document_similarity` is a score indicating the similarity between query and document where a higher score denotes greater similarity.
   * `$run_identifier` is an identifier of the run. This field is for your own convenience and has no purpose beyond bookkeeping.
   
For example, say we have two queries: `Q1` and `Q2` and we rank three documents (`DOC1`, `DOC2`, `DOC3`). For query `Q1`, we find the following similarity scores `score(Q1, DOC1) = 1.0`, `score(Q1, DOC2) = 0.5`, `score(Q1, DOC3) = 0.75`; and for `Q2`: `score(Q2, DOC1) = -0.1`, `score(Q2, DOC2) = 1.25`, `score(Q1, DOC3) = 0.0`. We can generate run using the following snippet:

In [44]:
import logging
import sys
import os

def write_run(model_name, data, out_f,
              max_objects_per_query=sys.maxsize,
              skip_sorting=False):
    """
    Write a run to an output file.
    Parameters:
        - model_name: identifier of run.
        - data: dictionary mapping topic_id to object_assesments;
            object_assesments is an iterable (list or tuple) of
            (relevance, object_id) pairs.
            The object_assesments iterable is sorted by decreasing order.
        - out_f: output file stream.
        - max_objects_per_query: cut-off for number of objects per query.
    """
    for subject_id, object_assesments in data.items():
        if not object_assesments:
            logging.warning('Received empty ranking for %s; ignoring.',
                            subject_id)

            continue

        # Probe types, to make sure everything goes alright.
        # assert isinstance(object_assesments[0][0], float) or \
        #     isinstance(object_assesments[0][0], np.float32)
        assert isinstance(object_assesments[0][1], str) or \
            isinstance(object_assesments[0][1], bytes)
        
        if not skip_sorting:
            object_assesments = sorted(object_assesments, reverse=True)

        if max_objects_per_query < sys.maxsize:
            object_assesments = object_assesments[:max_objects_per_query]

        if isinstance(subject_id, bytes):
            subject_id = subject_id.decode('utf8')

        for rank, (relevance, object_id) in enumerate(object_assesments):
            if isinstance(object_id, bytes):
                object_id = object_id.decode('utf8')

            out_f.write(
                '{subject} Q0 {object} {rank} {relevance} '
                '{model_name}\n'.format(
                    subject=subject_id,
                    object=object_id,
                    rank=rank + 1,
                    relevance=relevance,
                    model_name=model_name))
            
# The following writes the run to standard output.
# In your code, you should write the runs to local
# storage in order to pass them to trec_eval.
write_run(
    model_name='example',
    data={
        'Q1': ((11.0, 'DOC1'), (0.5, 'DOC2'), (0.75, 'DOC3')),
        'Q2': ((-0.1, 'DOC1'), (1.25, 'DOC2'), (0.0, 'DOC3')),
    },
    out_f=sys.stdout,
    max_objects_per_query=1000)

Q2 Q0 DOC2 1 1.25 example
Q2 Q0 DOC3 2 0.0 example
Q2 Q0 DOC1 3 -0.1 example
Q1 Q0 DOC1 1 11.0 example
Q1 Q0 DOC3 2 0.75 example
Q1 Q0 DOC2 3 0.5 example


In [45]:
#actually writing to example.run
fd = open('example.run','w+')
write_run(
    model_name='example',
    data={
        'Q1': ((1.0, 'DOC1'), (0.5, 'DOC2'), (0.75, 'DOC3')),
        'Q2': ((-0.1, 'DOC1'), (1.25, 'DOC2'), (0.0, 'DOC3')),
    },
    out_f=fd,
    max_objects_per_query=1000)
fd.close()

#actually writing to example.qrel
fd = open('example.qrel', 'w+')
fd.write("Q1 0 DOC1 1 \nQ1 0 DOC3 0 \nQ2 0 DOC3 1")
fd.close()

In [46]:
%%bash
#testing file content
echo 'run file:'
cat example.run
echo ''
echo 'query relevance file:'
cat example.qrel


run file:
Q2 Q0 DOC2 1 1.25 example
Q2 Q0 DOC3 2 0.0 example
Q2 Q0 DOC1 3 -0.1 example
Q1 Q0 DOC1 1 1.0 example
Q1 Q0 DOC3 2 0.75 example
Q1 Q0 DOC2 3 0.5 example

query relevance file:
Q1 0 DOC1 1 
Q1 0 DOC3 0 
Q2 0 DOC3 1

Now, imagine that we know that `DOC1` is relevant and `DOC3` is non-relevant for `Q1`. In addition, for `Q2` we only know of the relevance of `DOC3`. The query relevance file looks like:

    Q1 0 DOC1 1
    Q1 0 DOC3 0
    Q2 0 DOC3 1
    
We store the run and qrel in files `example.run` and `example.qrel` respectively on disk. We can now use TREC Eval to compute evaluation measures. In this example, we're only interested in Mean Average Precision and we'll only show this below for brevity. However, TREC Eval outputs much more information such as NDCG, recall, precision, etc.

    $ trec_eval -m all_trec -q example.qrel example.run | grep -E "^map\s"
    > map                   	Q1	1.0000
    > map                   	Q2	0.5000
    > map                   	all	0.7500
    
Now that we've discussed the output format of rankings and how you can compute evaluation measures from these rankings, we'll now proceed with an overview of the indexing framework you'll use.

In [47]:
%%bash
./trec_eval/trec_eval -m all_trec -q example.qrel example.run | grep -E "^ndcg_cut_10\s|^map_cut_1000\s|^P_5\s|^recall_1000\s"

P_5                   	Q1	0.2000
recall_1000           	Q1	1.0000
ndcg_cut_10           	Q1	1.0000
map_cut_1000          	Q1	1.0000
P_5                   	Q2	0.2000
recall_1000           	Q2	1.0000
ndcg_cut_10           	Q2	0.6309
map_cut_1000          	Q2	0.5000
P_5                   	all	0.2000
recall_1000           	all	1.0000
ndcg_cut_10           	all	0.8155
map_cut_1000          	all	0.7500


### Pyndri primer ###
For this assignment you will use [Pyndri](https://github.com/cvangysel/pyndri) [[1](https://arxiv.org/abs/1701.00749)], a python interface for [Indri](https://www.lemurproject.org/indri.php). We have indexed the document collection and you can query the index using Pyndri. We will start by giving you some examples of what Pyndri can do:

First we read the document collection index with Pyndri:

In [5]:
import pyndri

index = pyndri.Index('index/')

# If it crashes!!!
Nichita ( Based on what people said on Piazza ):

You can try this:

1. go to index/31/manifest
2. there's a line "<indri-distribution>Indri development release 5.8</indri-distribution>"

3. change "Indri development release 5.8" to "Indri release 5.11"

Run previous cell again

The loaded index can be used to access a collection of documents in an easy manner. We'll give you some examples to get some idea of what it can do, it is up to you to figure out how to use it for the remainder of the assignment.

First let's look at the number of documents, since Pyndri indexes the documents using incremental identifiers we can simply take the lowest index and the maximum document and consider the difference:

In [49]:
print("There are %d documents in this collection." % (index.maximum_document() - index.document_base()))

There are 164597 documents in this collection.


Let's take the first document out of the collection and take a look at it:

In [50]:
example_document = index.document(index.document_base())
print(example_document)

('AP890425-0001', (1360, 192, 363, 0, 880, 0, 200, 0, 894, 412, 92160, 3, 192, 0, 363, 34, 1441, 0, 174134, 0, 200, 0, 894, 412, 2652, 0, 810, 107, 49, 4903, 420, 0, 1, 48, 35, 489, 0, 35, 687, 192, 243, 0, 249311, 1877, 0, 1651, 1174, 0, 2701, 117, 412, 0, 810, 391, 245233, 1225, 5838, 16, 0, 233156, 3496, 0, 393, 17, 0, 2435, 4819, 930, 0, 0, 200, 0, 894, 0, 22, 398, 145, 0, 3, 271, 115, 0, 1176, 2777, 292, 0, 725, 192, 0, 0, 50046, 0, 1901, 1130, 0, 192, 0, 408, 0, 243779, 0, 0, 553, 192, 0, 363, 0, 3747, 0, 0, 0, 0, 1176, 0, 1239, 0, 0, 1115, 17, 0, 0, 585, 192, 1963, 0, 0, 412, 54356, 0, 773, 0, 0, 0, 192, 0, 0, 1130, 0, 363, 0, 545, 192, 0, 1174, 1901, 1130, 0, 4, 398, 145, 39, 0, 577, 0, 355, 0, 491, 0, 6025, 0, 0, 193156, 88, 34, 437, 0, 0, 1852, 0, 828, 0, 1588, 0, 0, 0, 2615, 0, 0, 107, 49, 420, 0, 0, 190, 7, 714, 2701, 0, 237, 192, 157, 0, 412, 34, 437, 0, 0, 200, 6025, 26, 0, 0, 0, 0, 363, 0, 22, 398, 145, 0, 200, 638, 126222, 6018, 0, 880, 0, 0, 161, 0, 0, 319, 894, 2701, 

Here we see a document consists of two things, a string representing the external document identifier and an integer list representing the identifiers of words that make up the document. Pyndri uses integer representations for words or terms, thus a token_id is an integer that represents a word whereas the token is the actual text of the word/term. Every id has a unique token and vice versa with the exception of stop words: words so common that there are uninformative, all of these receive the zero id.

To see what some ids and their matching tokens we take a look at the dictionary of the index:

In [7]:
token2id, id2token, _ = index.get_dictionary()
print(list(id2token.items())[:15])

[(1, 'new'), (2, 'percent'), (3, 'two'), (4, '1'), (5, 'people'), (6, 'million'), (7, '000'), (8, 'government'), (9, 'president'), (10, 'years'), (11, 'state'), (12, '2'), (13, 'states'), (14, 'three'), (15, 'time')]


Using this dictionary we can see the tokens for the (non-stop) words in our example document:

In [52]:
print([id2token[word_id] for word_id in example_document[1] if word_id > 0])

['52', 'students', 'arrested', 'takeover', 'university', 'massachusetts', 'building', 'fifty', 'two', 'students', 'arrested', 'tuesday', 'evening', 'occupying', 'university', 'massachusetts', 'building', 'overnight', 'protest', 'defense', 'department', 'funded', 'research', 'new', 'york', 'city', 'thousands', 'city', 'college', 'students', 'got', 'unscheduled', 'holiday', 'demonstrators', 'occupied', 'campus', 'administration', 'building', 'protest', 'possible', 'tuition', 'increases', 'prompting', 'officials', 'suspend', 'classes', '60', 'police', 'riot', 'gear', 'arrived', 'university', 'massachusetts', '5', 'p', 'm', 'two', 'hours', 'later', 'bus', 'drove', 'away', '29', 'students', 'camped', 'memorial', 'hall', 'students', 'charged', 'trespassing', '23', 'students', 'arrested', 'lying', 'bus', 'prevent', 'leaving', 'police', '300', 'students', 'stood', 'building', 'chanting', 'looking', 'students', 'hall', 'arrested', '35', 'students', 'occupied', 'memorial', 'hall', '1', 'p', 'm',

The reverse can also be done, say we want to look for news about the "University of Massachusetts", the tokens of that query can be converted to ids using the reverse dictionary:

In [53]:
query_tokens = index.tokenize("University of Massachusetts")
print("Query by tokens:", query_tokens)
query_id_tokens = [token2id.get(query_token,0) for query_token in query_tokens]
print("Query by ids with stopwords:", query_id_tokens)
query_id_tokens = [word_id for word_id in query_id_tokens if word_id > 0]
print("Query by ids without stopwords:", query_id_tokens)

Query by tokens: ['university', '', 'massachusetts']
Query by ids with stopwords: [200, 0, 894]
Query by ids without stopwords: [200, 894]


Naturally we can now match the document and query in the id space, let's see how often a word from the query occurs in our example document:

In [54]:
matching_words = sum([True for word_id in example_document[1] if word_id in query_id_tokens])
print("Document %s has %d word matches with query: \"%s\"." % (example_document[0], matching_words, ' '.join(query_tokens)))
print("Document %s and query \"%s\" have a %.01f%% overlap." % (example_document[0], ' '.join(query_tokens),matching_words/float(len(example_document[1]))*100))

Document AP890425-0001 has 13 word matches with query: "university  massachusetts".
Document AP890425-0001 and query "university  massachusetts" have a 2.5% overlap.


While this is certainly not everything Pyndri can do, it should give you an idea of how to use it. Please take a look at the [examples](https://github.com/cvangysel/pyndri) as it will help you a lot with this assignment.

**CAUTION**: Avoid printing out the whole index in this Notebook as it will generate a lot of output and is likely to corrupt the Notebook.

In [55]:
#Nichita:
#No good documentation found. Let's see the methods and attributes of an Index object:
methods = [method for method in dir(index) if '__' not in method and callable(getattr(index, method))]
for method in methods:
    print(method)

close
document
document_base
document_count
document_ids
document_length
ext_document_id
get_dictionary
get_term_frequencies
maximum_document
process_term
query
term_count
tokenize
total_terms
unique_terms


### Parsing the query file
You can parse the query file (`ap_88_89/topics_title`) using the following snippet:

In [10]:
import collections
import io
import logging
import sys

def parse_topics(file_or_files,
                 max_topics=sys.maxsize, delimiter=';'):
    assert max_topics >= 0 or max_topics is None

    topics = collections.OrderedDict()

    if not isinstance(file_or_files, list) and \
            not isinstance(file_or_files, tuple):
        if hasattr(file_or_files, '__iter__'):
            file_or_files = list(file_or_files)
        else:
            file_or_files = [file_or_files]

    for f in file_or_files:
        assert isinstance(f, io.IOBase)

        for line in f:
            assert(isinstance(line, str))

            line = line.strip()

            if not line:
                continue

            topic_id, terms = line.split(delimiter, 1)

            if topic_id in topics and (topics[topic_id] != terms):
                    logging.error('Duplicate topic "%s" (%s vs. %s).',
                                  topic_id,
                                  topics[topic_id],
                                  terms)

            topics[topic_id] = terms

            if max_topics > 0 and len(topics) >= max_topics:
                break

    return topics

with open('./ap_88_89/topics_title', 'r') as f_topics:
    print(parse_topics([f_topics]))

OrderedDict([('51', 'Airbus Subsidies'), ('52', 'South African Sanctions'), ('53', 'Leveraged Buyouts'), ('54', 'Satellite Launch Contracts'), ('55', 'Insider Trading'), ('56', 'Prime (Lending) Rate Moves, Predictions'), ('57', 'MCI'), ('58', 'Rail Strikes'), ('59', 'Weather Related Fatalities'), ('60', 'Merit-Pay vs. Seniority'), ('61', 'Israeli Role in Iran-Contra Affair'), ('62', "Military Coups D'etat"), ('63', 'Machine Translation'), ('64', 'Hostage-Taking'), ('65', 'Information Retrieval Systems'), ('66', 'Natural Language Processing'), ('67', 'Politically Motivated Civil Disturbances'), ('68', 'Health Hazards from Fine-Diameter Fibers'), ('69', 'Attempts to Revive the SALT II Treaty'), ('70', 'Surrogate Motherhood'), ('71', 'Border Incursions'), ('72', 'Demographic Shifts in the U.S.'), ('73', 'Demographic Shifts across National Boundaries'), ('74', 'Conflicting Policy'), ('75', 'Automation'), ('76', 'U.S. Constitution - Original Intent'), ('77', 'Poaching'), ('78', 'Greenpeace'

### Task 1: Implement and compare lexical IR methods [35 points] ### 

In this task you will implement a number of lexical methods for IR using the **Pyndri** framework. Then you will evaluate these methods on the dataset we have provided using **TREC Eval**.

Use the **Pyndri** framework to get statistics of the documents (term frequency, document frequency, collection frequency; **you are not allowed to use the query functionality of Pyndri**) and implement the following scoring methods in **Python**:

- [TF-IDF](http://nlp.stanford.edu/IR-book/html/htmledition/tf-idf-weighting-1.html) and 
- [BM25](http://nlp.stanford.edu/IR-book/html/htmledition/okapi-bm25-a-non-binary-model-1.html) with k1=1.2 and b=0.75. **[5 points]**
- Language models ([survey](https://drive.google.com/file/d/0B-zklbckv9CHc0c3b245UW90NE0/view))
    - Jelinek-Mercer (explore different values of 𝛌 in the range [0.1, 0.5, 0.9]). **[5 points]**
    - Dirichlet Prior (explore different values of 𝛍 [500, 1000, 1500]). **[5 points]**
    - Absolute discounting (explore different values of 𝛅 in the range [0.1, 0.5, 0.9]). **[5 points]**
    - [Positional Language Models](http://sifaka.cs.uiuc.edu/~ylv2/pub/sigir09-plm.pdf) define a language model for each position of a document, and score a document based on the scores of its PLMs. The PLM is estimated based on propagated counts of words within a document through a proximity-based density function, which both captures proximity heuristics and achieves an effect of “soft” passage retrieval. Implement the PLM, all five kernels, but only the Best position strategy to score documents. Use 𝛔 equal to 50, and Dirichlet smoothing with 𝛍 optimized on the validation set (decide how to optimize this value yourself and motivate your decision in the report). **[10 points]**
    
Implement the above methods and report evaluation measures (on the test set) using the hyper parameter values you optimized on the validation set (also report the values of the hyper parameters). Use TREC Eval to obtain the results and report on `NDCG@10`, Mean Average Precision (`MAP@1000`), `Precision@5` and `Recall@1000`.

For the language models, create plots showing `NDCG@10` with varying values of the parameters. You can do this by chaining small scripts using shell scripting (preferred) or execute trec_eval using Python's `subprocess`.

Compute significance of the results using a [two-tailed paired Student t-test](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html) **[5 points]**. Be wary of false rejection of the null hypothesis caused by the [multiple comparisons problem](https://en.wikipedia.org/wiki/Multiple_comparisons_problem). There are multiple ways to mitigate this problem and it is up to you to choose one.

Analyse the results by identifying specific queries where different methods succeed or fail and discuss possible reasons that cause these differences. This is *very important* in order to understand who the different retrieval functions behave.

**NOTE**: Don’t forget to use log computations in your calculations to avoid underflows. 

**IMPORTANT**: You should structure your code around the helper functions we provide below.

In [22]:
import time
import numpy as np
from collections import defaultdict

In [24]:
with open('./ap_88_89/topics_title', 'r') as f_topics:
    queries = parse_topics([f_topics])

index = pyndri.Index('index/')

num_documents = index.maximum_document() - index.document_base()

dictionary = pyndri.extract_dictionary(index)

tokenized_queries = {
    query_id: [dictionary.translate_token(token)
               for token in index.tokenize(query_string)
               if dictionary.has_token(token)]
    for query_id, query_string in queries.items()}

query_term_ids = set(
    query_term_id
    for query_term_ids in tokenized_queries.values()
    for query_term_id in query_term_ids)

print('Gathering statistics about', len(query_term_ids), 'terms.')

# inverted index creation.
start_time = time.time()

document_lengths = {}
unique_terms_per_document = {}

inverted_index = defaultdict(dict)
collection_frequencies = defaultdict(int)

total_terms = 0

for int_doc_id in range(index.document_base(), index.maximum_document()):
    ext_doc_id, doc_token_ids = index.document(int_doc_id)

    document_bow = collections.Counter(
        token_id for token_id in doc_token_ids
        if token_id > 0)
    document_length = sum(document_bow.values())

    document_lengths[int_doc_id] = document_length
    total_terms += document_length

    unique_terms_per_document[int_doc_id] = len(document_bow)
    
    
    for query_term_id in query_term_ids:
        assert query_term_id is not None

        document_term_frequency = document_bow.get(query_term_id, 0)

        if document_term_frequency == 0:
            continue

        collection_frequencies[query_term_id] += document_term_frequency
        inverted_index[query_term_id][int_doc_id] = document_term_frequency
        
avg_doc_length = total_terms / num_documents

print('Inverted index creation took', time.time() - start_time, 'seconds.')

Gathering statistics about 456 terms.
Inverted index creation took 25.708834886550903 seconds.


In [25]:
def run_retrieval(model_name, score_fn, score_fn_params_dict):
    """
    Runs a retrieval method for all the queries and writes the TREC-friendly results in a file.
    
    :param model_name: the name of the model (a string)
    :param score_fn: the scoring function (a function - see below for an example) 
    """
    run_out_path = '{}.run'.format(model_name)

    if os.path.exists(run_out_path):
        return

    retrieval_start_time = time.time()

    print('Retrieving using', model_name)

    data = {}
    
    for query_id, query_token_list in tokenized_queries.items():
        #for each query
        
        #print('query', [dictionary.id2token[x] for x in query_token_list])
        query_docs_score_list = []
        for int_doc_id in range(index.document_base(), index.maximum_document()):
            #for each document
            
            ext_doc_id, doc_terms_ids = index.document(int_doc_id)
            
            q_d_score = multinomial_scoring_method_document_query(score_fn, score_fn_params_dict, int_doc_id, query_token_list)

            query_docs_score_list.append((q_d_score, ext_doc_id))
                
        data[query_id] = query_docs_score_list
        
    with open(run_out_path, 'w') as f_out:
        write_run(
            model_name=model_name,
            data=data,
            out_f=f_out,
            max_objects_per_query=1000)

In [ ]:
def multinomial_scoring_method_document_query(score_fn, score_fn_params_dict, int_doc_id, query_token_list):
    #cosine similarity could be used between the two
    #teacher suggested addition because it is more efficient
    score = 0
    
    for token_id in query_token_list:
        score_fn_params_dict['int_document_id'] = int_doc_id
        score_fn_params_dict['query_term_id'] = token_id 
        score_fn_params_dict['document_term_freq'] = None
        score += score_fn(**score_fn_params_dict)
        
    return score

In [ ]:
def tfidf(int_document_id, query_term_id, document_term_freq):
    """
    Scoring function for a document and a query term
    
    :param int_document_id: the document id
    :param query_token_id: the query term id (assuming you have split the query to tokens)
    :param document_term_freq: the document term frequency of the query term 
    
    tf_idf(t;d)=log(1+tf(t;d)) * log(n/df(t))
    former log expresses how much is d about t
    latter log expresses inverse of how many documents are about t
    used the log(1+tf) in order to dampen the result and to not overflow
    tf is not normalized, but could be
    """
    
    #also is document_term_freq == inverted_index[query_term_id][int_document_id] ??
    #I am using inverted_index[query_term_id][int_document_id] atm because I strongly think it is
    if int_document_id not in inverted_index[query_term_id].keys():
        tf = 0
    else:
        tf = inverted_index[query_term_id][int_document_id]
    
    if len(inverted_index[query_term_id]) == 0:
        #making sure inverted index is never infinity
        idf = 0
    else:
        idf = np.log2(num_documents)-np.log2(len(inverted_index[query_term_id]))
    
    score = np.log2(1 + tf) * idf
    #if score != 0:
    #    #print(score, tf, len(inverted_index[query_term_id]), dictionary.id2token[query_term_id])
    return score

# combining the three functions above:
start_time = time.time()
run_retrieval('tfidf', tfidf, None)
print('tfidf on all query-docs took: ' + str(time.time()-start_time))
# TODO implement the rest of the retrieval functions 

# TODO implement tools to help you with the analysis of the results.

In [137]:
def bm25(int_document_id, query_term_id, document_term_freq, k1, b):
    """
    Scoring function for a document and a query term
    
    :param int_document_id: the document id
    :param query_token_id: the query term id (assuming you have split the query to tokens)
    :param document_term_freq: the document term frequency of the query term 
    
    bm25(t;d) = (k1+1)*tf(t;d) / (tf(t;d) + k1*((1-b) + b*(len(d)/avg_len_d)))
    
    – Most widely used weighting in IR
    – Has TF, IDF, and document length components
    – But only loosely inspired by probabilistic model
    """
    
    #also is document_term_freq == inverted_index[query_term_id][int_document_id] ??
    #I am using inverted_index[query_term_id][int_document_id] atm because I strongly think it is
    
    if int_document_id not in inverted_index[query_term_id].keys():
        tf = 0
    else:
        tf = inverted_index[query_term_id][int_document_id]
    
    score = (k1+1)*tf / (tf + k1*((1-b) + b*(document_lengths[int_document_id]/avg_doc_length)))
    return score

start_time = time.time()
run_retrieval('bm25', bm25, {'k1':1.2, 'b':0.75})
print('bm25 on all query-docs took: ' + str(time.time()-start_time))

Retrieving using bm25
bm25 on all query-docs took: 519.0541322231293


# Language models: 
We are interested in estimating p(d|q) in order to get the most probable document, given a query

$ p(d|q) = \frac{p(q|d)p(d)}{p(q)} \propto \\
\propto p(q|d)p(d) $

Also, using the assumption that our prior on the documents is uniform

$ p(q|d)p(d) = p(q|d) \\
\Rightarrow p(d|q) \propto p(q|d) $

Assuming a multinomial model for the query language model of the query, aka unigram:

$p(q|d) = \prod_{w_i \in q} p(w_i|d)$

We need to have the language model that generated d in order to be able to compute the above value:
Each document, d has a distribution $\theta_d$ which generated it.
The equation now is:

$p(q|d) = \prod_{w_i \in q} (w_i|\theta_d)$

We approximate $\theta_d$ to the probability distribution most likely to have generated d

$\hat{\theta_d} = \underset{\theta_d}{argmax}p(d|\theta_d)$

Given a multinomial assumption for $\theta_d$

$p(d|\theta_d) $ $= \underset{w \in V}{\prod}p(w|\theta_d)^{tf(w;d)} \\
                  = \underset{w \in d}{\prod}p(w|\theta_d)$

Now by using this new $p(d|\theta_d)$ and applying log we get the log likelihood: 

$ log p(d|\theta_d) = \underset{w \in d}{\sum}tf(w;d)p(w|\theta_d)$

In order to get the maximum likelihood we need to following a dual Lagrangian as we need to estimate $\theta_d$ and keep the probability of words to sum to 1:

$L = log p(d|\theta_d) + \lambda(1-\underset{w \in V}{\sum}p(w|\theta_d))$

By taking the derivatives we get to:

$p_{ml}(w|\theta_d)=\frac{tf(w; d)}{|d|}$

Now the generative process of generating a query is:

$ p(q|\theta_d) = \underset{w \in q}{\prod}{\frac{tf(w; d)}{|d|}} $

# Smoothing
Smoothing means adjusting the ML estimation to avoid 0 probability and it is dont by taking mass from all the seen words and distributing it to the unseen words. This is similar to having a prior on unseen documents.

### Additive smoothing
Assumes that every word, even the unseen ones, have a small probability $\epsilon$ <br>
$p_\epsilon(w|\theta_d) = \frac{tf(w;d)+\epsilon}{|d|+\epsilon V}$ <br>

### Better smoothing..
The probability of an unseen word should be proportional to the probability of a word given a background model.
* language model estimated based on the entire document collection, C

Estimate p(w|C):
* words contributing equally: $ p(w|C) = \frac{tf(w;C)}{|C|} $
* documents contributing equally: $ p(w|C) = \frac{1}{|C|} \sum_{d \in C}\frac{tf(w;d)}{|d|} $
* document frequency $ p(w|C) = \frac{df(w)}{\sum_{w' \in V} df(w)} $

## Jelinek-Mercer smoothing, aka: Linearly interpolate with background language model ( $\lambda$ ) <br>
$ \hat{p}_\lambda(w|d) $ $= \lambda p(w|d) + (1-\lambda) p(w|C)\\
= \lambda \frac{tf(w;d)}{|d|} + (1-\lambda) \frac{tf(w;C)}{|C|}$

# TODO rerun language models as you were stupid and recommit data

In [78]:
def jelinek_mercer_smoothing(int_document_id, query_term_id, document_term_freq, lambda_jm):
    """
    Scoring function for a document and a query term
    
    :param int_document_id: the document id
    :param query_token_id: the query term id (assuming you have split the query to tokens)
    :param document_term_freq: the document term frequency of the query term 
    
    j_m(t;d) = lambda*tf(w;d)/|d| + (1-lambda)*tf(w;C)/|C|
    """
    
    #also is document_term_freq == inverted_index[query_term_id][int_document_id] ??
    #I am using inverted_index[query_term_id][int_document_id] atm because I strongly think it is
    
    if int_document_id not in inverted_index[query_term_id].keys():
        tf = 0
    else:
        tf = inverted_index[query_term_id][int_document_id]
    
    if query_term_id not in collection_frequencies.keys():
        tf_collection = 0
    else:
        tf_collection = collection_frequencies[query_term_id]
    
    if document_lengths[int_document_id] == 0:
        #when the document has nothing in it only the background model will influence score
        tf = 0
        doc_len = 1
    else:
        doc_len = document_lengths[int_document_id]
    
    score = lambda_jm*tf / doc_len + (1-lambda_jm)*tf_collection/total_terms
    print('score= ', score, ' p(w|C)= ', tf_collection/total_terms)
    return score


#try 0.1 0.5 0.9
for lambda_jm in [0.1, 0.5, 0.9]:
    start_time = time.time()
    run_retrieval('test_jelinek_mercer:lambda='+str(lambda_jm), jelinek_mercer_smoothing, {'lambda_jm':lambda_jm})
    print('jelinek mercer query-docs took: ' + str(time.time()-start_time))

Retrieving using test_jelinek_mercer:lambda=0.1
score=  0.00015315943122784503  p(w|C)=  0.00017017714580871672
score=  2.5842855443150243e-05  p(w|C)=  2.8714283825722493e-05
score=  0.0004551498286216874  p(w|C)=  0.0005057220318018748
score=  0.0002103463440154102  p(w|C)=  0.00023371816001712244
score=  0.00015315943122784503  p(w|C)=  0.00017017714580871672
score=  2.5842855443150243e-05  p(w|C)=  2.8714283825722493e-05
score=  0.0004551498286216874  p(w|C)=  0.0005057220318018748
score=  0.0002103463440154102  p(w|C)=  0.00023371816001712244
score=  0.00015315943122784503  p(w|C)=  0.00017017714580871672
score=  2.5842855443150243e-05  p(w|C)=  2.8714283825722493e-05
score=  0.0004551498286216874  p(w|C)=  0.0005057220318018748
score=  0.0002103463440154102  p(w|C)=  0.00023371816001712244
score=  0.00015315943122784503  p(w|C)=  0.00017017714580871672
score=  2.5842855443150243e-05  p(w|C)=  2.8714283825722493e-05
score=  0.0004551498286216874  p(w|C)=  0.0005057220318018748
sco

score=  0.0002103463440154102  p(w|C)=  0.00023371816001712244
score=  0.00015315943122784503  p(w|C)=  0.00017017714580871672
score=  2.5842855443150243e-05  p(w|C)=  2.8714283825722493e-05
score=  0.0004551498286216874  p(w|C)=  0.0005057220318018748
score=  0.0002103463440154102  p(w|C)=  0.00023371816001712244
score=  0.00015315943122784503  p(w|C)=  0.00017017714580871672
score=  2.5842855443150243e-05  p(w|C)=  2.8714283825722493e-05
score=  0.0004551498286216874  p(w|C)=  0.0005057220318018748
score=  0.0002103463440154102  p(w|C)=  0.00023371816001712244
score=  0.00015315943122784503  p(w|C)=  0.00017017714580871672
score=  2.5842855443150243e-05  p(w|C)=  2.8714283825722493e-05
score=  0.0004551498286216874  p(w|C)=  0.0005057220318018748
score=  0.0002103463440154102  p(w|C)=  0.00023371816001712244
score=  0.00015315943122784503  p(w|C)=  0.00017017714580871672
score=  2.5842855443150243e-05  p(w|C)=  2.8714283825722493e-05
score=  0.0004551498286216874  p(w|C)=  0.00050572

score=  0.00015315943122784503  p(w|C)=  0.00017017714580871672
score=  2.5842855443150243e-05  p(w|C)=  2.8714283825722493e-05
score=  0.0004551498286216874  p(w|C)=  0.0005057220318018748
score=  0.0002103463440154102  p(w|C)=  0.00023371816001712244
score=  0.00015315943122784503  p(w|C)=  0.00017017714580871672
score=  2.5842855443150243e-05  p(w|C)=  2.8714283825722493e-05
score=  0.0004551498286216874  p(w|C)=  0.0005057220318018748
score=  0.0002103463440154102  p(w|C)=  0.00023371816001712244
score=  0.00015315943122784503  p(w|C)=  0.00017017714580871672
score=  2.5842855443150243e-05  p(w|C)=  2.8714283825722493e-05
score=  0.0004551498286216874  p(w|C)=  0.0005057220318018748
score=  0.0002103463440154102  p(w|C)=  0.00023371816001712244
score=  0.00015315943122784503  p(w|C)=  0.00017017714580871672
score=  2.5842855443150243e-05  p(w|C)=  2.8714283825722493e-05
score=  0.0004551498286216874  p(w|C)=  0.0005057220318018748
score=  0.0002103463440154102  p(w|C)=  0.000233718

score=  0.0004551498286216874  p(w|C)=  0.0005057220318018748
score=  0.0002103463440154102  p(w|C)=  0.00023371816001712244
score=  0.00015315943122784503  p(w|C)=  0.00017017714580871672
score=  2.5842855443150243e-05  p(w|C)=  2.8714283825722493e-05
score=  0.0004551498286216874  p(w|C)=  0.0005057220318018748
score=  0.0002103463440154102  p(w|C)=  0.00023371816001712244
score=  0.00015315943122784503  p(w|C)=  0.00017017714580871672
score=  2.5842855443150243e-05  p(w|C)=  2.8714283825722493e-05
score=  0.0004551498286216874  p(w|C)=  0.0005057220318018748
score=  0.0002103463440154102  p(w|C)=  0.00023371816001712244
score=  0.00015315943122784503  p(w|C)=  0.00017017714580871672
score=  2.5842855443150243e-05  p(w|C)=  2.8714283825722493e-05
score=  0.0004551498286216874  p(w|C)=  0.0005057220318018748
score=  0.0002103463440154102  p(w|C)=  0.00023371816001712244
score=  0.00015315943122784503  p(w|C)=  0.00017017714580871672
score=  2.5842855443150243e-05  p(w|C)=  2.871428382

score=  0.00015315943122784503  p(w|C)=  0.00017017714580871672
score=  2.5842855443150243e-05  p(w|C)=  2.8714283825722493e-05
score=  0.0004551498286216874  p(w|C)=  0.0005057220318018748
score=  0.0002103463440154102  p(w|C)=  0.00023371816001712244
score=  0.00015315943122784503  p(w|C)=  0.00017017714580871672
score=  2.5842855443150243e-05  p(w|C)=  2.8714283825722493e-05
score=  0.0011850768359209576  p(w|C)=  0.0005057220318018748
score=  0.0002103463440154102  p(w|C)=  0.00023371816001712244
score=  0.00015315943122784503  p(w|C)=  0.00017017714580871672
score=  2.5842855443150243e-05  p(w|C)=  2.8714283825722493e-05
score=  0.0004551498286216874  p(w|C)=  0.0005057220318018748
score=  0.0002103463440154102  p(w|C)=  0.00023371816001712244
score=  0.00015315943122784503  p(w|C)=  0.00017017714580871672
score=  2.5842855443150243e-05  p(w|C)=  2.8714283825722493e-05
score=  0.0004551498286216874  p(w|C)=  0.0005057220318018748
score=  0.0002103463440154102  p(w|C)=  0.000233718

...

score=  0.00015315943122784503  p(w|C)=  0.00017017714580871672
score=  2.5842855443150243e-05  p(w|C)=  2.8714283825722493e-05
score=  0.0004551498286216874  p(w|C)=  0.0005057220318018748
score=  0.0002103463440154102  p(w|C)=  0.00023371816001712244
score=  0.00045990789748551376  p(w|C)=  0.00017017714580871672
score=  2.5842855443150243e-05  p(w|C)=  2.8714283825722493e-05
score=  0.0004551498286216874  p(w|C)=  0.0005057220318018748
score=  0.0002103463440154102  p(w|C)=  0.00023371816001712244
score=  0.00015315943122784503  p(w|C)=  0.00017017714580871672
score=  2.5842855443150243e-05  p(w|C)=  2.8714283825722493e-05
score=  0.0004551498286216874  p(w|C)=  0.0005057220318018748
score=  0.0002103463440154102  p(w|C)=  0.00023371816001712244
score=  0.00015315943122784503  p(w|C)=  0.00017017714580871672
score=  2.5842855443150243e-05  p(w|C)=  2.8714283825722493e-05
score=  0.0004551498286216874  p(w|C)=  0.0005057220318018748
score=  0.0002103463440154102  p(w|C)=  0.000233718

score=  2.5842855443150243e-05  p(w|C)=  2.8714283825722493e-05
score=  0.0004551498286216874  p(w|C)=  0.0005057220318018748
score=  0.0002103463440154102  p(w|C)=  0.00023371816001712244
score=  0.00015315943122784503  p(w|C)=  0.00017017714580871672
score=  2.5842855443150243e-05  p(w|C)=  2.8714283825722493e-05
score=  0.0004551498286216874  p(w|C)=  0.0005057220318018748
score=  0.0002103463440154102  p(w|C)=  0.00023371816001712244
score=  0.00015315943122784503  p(w|C)=  0.00017017714580871672
score=  2.5842855443150243e-05  p(w|C)=  2.8714283825722493e-05
score=  0.0004551498286216874  p(w|C)=  0.0005057220318018748
score=  0.0002103463440154102  p(w|C)=  0.00023371816001712244
score=  0.00015315943122784503  p(w|C)=  0.00017017714580871672
score=  2.5842855443150243e-05  p(w|C)=  2.8714283825722493e-05
score=  0.0004551498286216874  p(w|C)=  0.0005057220318018748
score=  0.0002103463440154102  p(w|C)=  0.00023371816001712244
score=  0.00015315943122784503  p(w|C)=  0.000170177

score=  0.0004551498286216874  p(w|C)=  0.0005057220318018748
score=  0.0002103463440154102  p(w|C)=  0.00023371816001712244
score=  0.00015315943122784503  p(w|C)=  0.00017017714580871672
score=  2.5842855443150243e-05  p(w|C)=  2.8714283825722493e-05
score=  0.0004551498286216874  p(w|C)=  0.0005057220318018748
score=  0.0002103463440154102  p(w|C)=  0.00023371816001712244
score=  0.00015315943122784503  p(w|C)=  0.00017017714580871672
score=  2.5842855443150243e-05  p(w|C)=  2.8714283825722493e-05
score=  0.0004551498286216874  p(w|C)=  0.0005057220318018748
score=  0.0002103463440154102  p(w|C)=  0.00023371816001712244
score=  0.00015315943122784503  p(w|C)=  0.00017017714580871672
score=  2.5842855443150243e-05  p(w|C)=  2.8714283825722493e-05
score=  0.0004551498286216874  p(w|C)=  0.0005057220318018748
score=  0.0002103463440154102  p(w|C)=  0.00023371816001712244
score=  0.00015315943122784503  p(w|C)=  0.00017017714580871672
score=  2.5842855443150243e-05  p(w|C)=  2.871428382

score=  0.00015315943122784503  p(w|C)=  0.00017017714580871672
score=  2.5842855443150243e-05  p(w|C)=  2.8714283825722493e-05
score=  0.0004551498286216874  p(w|C)=  0.0005057220318018748
score=  0.0002103463440154102  p(w|C)=  0.00023371816001712244
score=  0.00015315943122784503  p(w|C)=  0.00017017714580871672
score=  2.5842855443150243e-05  p(w|C)=  2.8714283825722493e-05
score=  0.0004551498286216874  p(w|C)=  0.0005057220318018748
score=  0.0002103463440154102  p(w|C)=  0.00023371816001712244
score=  0.00015315943122784503  p(w|C)=  0.00017017714580871672
score=  2.5842855443150243e-05  p(w|C)=  2.8714283825722493e-05
score=  0.0004551498286216874  p(w|C)=  0.0005057220318018748
score=  0.0002103463440154102  p(w|C)=  0.00023371816001712244
score=  0.00015315943122784503  p(w|C)=  0.00017017714580871672
score=  2.5842855443150243e-05  p(w|C)=  2.8714283825722493e-05
score=  0.0004551498286216874  p(w|C)=  0.0005057220318018748
score=  0.0002103463440154102  p(w|C)=  0.000233718

KeyboardInterrupt: 

## Absolute discounting smoothing ($\delta$)
IDEA:
* Lower the probability of seen words by subtracting a constant from their counts

In [121]:
def absolute_discounting(int_document_id, query_term_id, document_term_freq, delta):
    """
    Scoring function for a document and a query term
    
    :param int_document_id: the document id
    :param query_token_id: the query term id (assuming you have split the query to tokens)
    :param document_term_freq: the document term frequency of the query term 
    
    a_d(t;d) = max(tf(w;d)-delta, 0) / |d| + delta*|d_unique|/|d| * tf(w;C)/|C|
    """
    
    #also is document_term_freq == inverted_index[query_term_id][int_document_id] ??
    #I am using inverted_index[query_term_id][int_document_id] atm because I strongly think it is
    
    if int_document_id not in inverted_index[query_term_id].keys():
        tf = 0
    else:
        tf = inverted_index[query_term_id][int_document_id]
    
    if query_term_id not in collection_frequencies.keys():
        tf_collection = 0
    else:
        tf_collection = collection_frequencies[query_term_id]
    
    if document_lengths[int_document_id] == 0:
        #when the document has nothing in it only the background model will influence score
        tf = delta
        doc_len = 1
    else:
        doc_len = document_lengths[int_document_id]
    unique_terms_count = unique_terms_per_document[int_document_id]
    score = max(tf-delta,0)/doc_len + delta* unique_terms_count /doc_len*tf_collection/total_terms
    return score

for delta in [0.1, 0.5, 0.9]:
    start_time = time.time()
    run_retrieval('absolute_discounting:delta='+str(delta), absolute_discounting, {'delta':delta})
    print('absolute discounting all query-docs took: ' + str(time.time()-start_time))

absolute discounting all query-docs took: 2.86102294921875e-05
Retrieving using absolute_discounting:delta=0.5
absolute discounting all query-docs took: 560.0043470859528
Retrieving using absolute_discounting:delta=0.9
absolute discounting all query-docs took: 554.1321775913239


TODO COMMENT MULTINOMIAL DIRICHLET 
ALSO REASON for choosing p(w|C) = tf(w;C)/|C| everywhere

In [95]:
def dirichlet_prior_smoothing(int_document_id, query_term_id, document_term_freq, mu):
    """
    Scoring function for a document and a query term
    
    :param int_document_id: the document id
    :param query_token_id: the query term id (assuming you have split the query to tokens)
    :param document_term_freq: the document term frequency of the query term 
    
    d(t;d) = (tf(w;d) + mu + tf(w;C)/|C|) / (|d| + mu)
    """
    
    #also is document_term_freq == inverted_index[query_term_id][int_document_id] ??
    #I am using inverted_index[query_term_id][int_document_id] atm because I strongly think it is
    if int_document_id not in inverted_index[query_term_id].keys():
        tf = 0
    else:
        tf = inverted_index[query_term_id][int_document_id]
    
    if query_term_id not in collection_frequencies.keys():
        tf_collection = 0
    else:
        tf_collection = collection_frequencies[query_term_id]
    
    doc_len = document_lengths[int_document_id]
    
    score = (tf+mu*tf_collection/total_terms ) / ( doc_len + mu )
    return score


for mu_value in [500, 1000, 1500]:
    start_time = time.time()
    run_retrieval('dirichlet_prior:mu='+str(mu_value), dirichlet_prior_smoothing, {'mu':mu_value})
    print('dirichlet prior on query-docs took: ' + str(time.time()-start_time))

Retrieving using dirichlet_prior:mu=500
dirichlet prior on query-docs took: 491.2328951358795
Retrieving using dirichlet_prior:mu=1000
dirichlet prior on query-docs took: 491.3247084617615
Retrieving using dirichlet_prior:mu=1500
dirichlet prior on query-docs took: 489.79303646087646


Positional Language Models define a language model for each position of a document, and score a document based on the scores of its PLMs. The PLM is estimated based on propagated counts of words within a document through a proximity-based density function, which both captures proximity heuristics and achieves an effect of “soft” passage retrieval. Implement the PLM, all five kernels, but only the Best position strategy to score documents. Use 𝛔 equal to 50, and Dirichlet smoothing with 𝛍 optimized on the validation set (decide how to optimize this value yourself and motivate your decision in the report). [10 points]

"3.2.1    Best Position Strategy
Our first strategy is to simply score a document based on
the score of its best matching position, formally,

$S(q,d)=\underset{i \in [1,N]}{max}\{S(q,d,i)\}$

This strategy resembles most existing studies on passage
retrieval, which generally considered evidences from the best
matching passage [4, 9, 16]"

## TODO finish PLM
## Positional Language Model
Defines a language model for each position of a document. Moreover, model should be a fuzzy description of the paragrpah, centered at the given position.

term-frequency per position:

$tf'(w,j;d)=\sum_{i=0}^{|d|}tf(w,j;d)k(j,i) $ <br>
where k is a non icreasing function describing the propagation of a term in it's vecinity <br>
examples of k: <br>
* A constant function: <br>
$ k(i,j) $$= 1, |i-j| <= range \\
           = 0, otherwise$ 
* or a Gaussian: <br>
$ k(i,j)= exp \left( \frac{-(i-j)^2}{2\sigma^2} \right)$

As a result we have the following language model per position: <br>

$p(w|d,i) = \frac{tf'(w,i;d)k(j,i)}{\sum_{w' \in V} tf'(w', i;d)} $

This can result in multiple representations of d. As a result we design a per query score for each PLM:<br>
$S(q,d,i)=-\sum_{w \in V}p(w|q)\frac{p(w|q)}{p(w|d,i)}$, <br>
where p(w|q) is an estimated query language model

In [60]:
def get_tfifd_top_n_doc_query_pair(n):
    dic = {}
    
    counter = 0
    previous_query_id = -1
    with open('tfidf.run','r') as fd:
        for line in fd:
            q_id, _, ext_doc_id = line.strip().split(' ')[0:3]
            int_doc_id = index.document_ids([ext_doc_id])[0][1]
            
            if previous_query_id != q_id:
                #if new query
                previous_query_id = q_id
                counter = 0

            if counter < n:
                #add it if it's in top n
                if int_doc_id not in dic.keys():
                     dic[int_doc_id] = []
                dic[int_doc_id].append(q_id)
            else:
                #skip if not in top n
                pass
            counter += 1
            
    return dic

start_time = time.time()
top_n = 100
doc_query_top_n_dic = get_tfifd_top_n_doc_query_pair(top_n)
print('creating top ' + str(top_n) + ' doc to query dict took:' + str(time.time()-start_time))

creating top 100 doc to query dict took:1.3339924812316895


In [61]:
from scipy.stats import norm

In [62]:
'''
def gaussian_cdf(i, sigma, N):
    #AKA Z_i
    #N is the document length
    #sum_j in 1,N} k(j,i) = np.sqrt(2*np.pi*sigma*sigma) * (phi((N-i)/sigma) - phi((i-1)/sigma))
    #phi = cummulative normal distribution 
    cdfs = norm.cdf((N-i)/sigma) - norm.cdf((1-i)/sigma)
    if cdfs == 0:
        print('ahoo its cdfs', norm.cdf((N-i)/sigma), N, i)
    res = np.sqrt(2*np.pi*sigma*sigma) * (norm.cdf((N-i)/sigma) - norm.cdf((1-i)/sigma))
    if res == 0:
        print('HERE', sigma, N, i)
    return res
'''
def gaussian_kernel(absolute_difference, sigma):
    # absolute_difference = abs(i-j)
    return np.exp( (-(absolute_difference)*(absolute_difference)) / (2*sigma*sigma) )

# sigma*np.sqrt(2*np.pi) * Cumulative normal distribution function at x
def GaussianCDF(x, mean, sigma):
    res = 0
    x=(x - mean) / sigma;
    if x == 0:
        res=0.5
    else:
        oor2pi = 1/(np.sqrt((2) * np.pi));
        t = 1 / ((1) + 0.2316419 * abs(x))
        t *= oor2pi * np.exp(-0.5 * x * x) \
             * (0.31938153   + t \
             * (-0.356563782 + t \
             * (1.781477937  + t \
             * (-1.821255978 + t * 1.330274429))))
        if x >= 0:
            res = 1 - t
        else:
            res = t
    return sigma*np.sqrt(2*np.pi)*res


def triangle_kernel(absolute_difference, sigma):
    # absolute_difference = abs(i-j)
    ans = 0
    if absolute_difference <= sigma:
        ans = 1 - absolute_difference / sigma
    return ans

#Cumulative distribution function of Triangle Kernel
def TriangleCDF( x,  mean,  sigma):
    res = 0
    x = (x - mean) / sigma
    if x >= 1:
        res = sigma;
    elif x < -1:
        res = 0
    elif x < 0:
        res = sigma * (1 - abs(x)) * (1 - abs(x)) / 2.0
    else:
        res = sigma - sigma * (1 - x) * (1 - x) / 2.0
    return res

def cosine_kernel(absolute_difference, sigma):
    # absolute_difference = abs(i-j)
    ans = 0
    if absolute_difference <= sigma:
        ans = 1/2*(1+np.cos(absolute_difference*np.pi/sigma))
    return ans

#Cumulative distribution function of Cosine Kernel
def CosineCDF( x,  mean,  sigma):
    res = 0
    x = (x - mean) / sigma
    if x >= 1:
        res = sigma
    elif x < -1:
        res = 0
    elif x < 0:
        res = sigma * (1 + x -  np.sin(np.pi * x) / np.pi) / 2.0
    else:
        res = sigma - sigma * (1 - x + np.sin(np.pi * x) / np.pi) / 2.0
    return res

def circle_kernel(absolute_difference, sigma):
    # absolute_difference = abs(i-j)
    ans = 0
    if absolute_difference <= sigma:
        ans = np.sqrt(1-(absolute_difference/sigma)*(absolute_difference/sigma))
    return ans

#Cumulative distribution function of Circle Kernel
def CircleCDF( x,  mean,  sigma):
    res = 0
    x = (x - mean) / sigma
    if x >= 1:
        res = (np.pi - 2.0) * sigma
    elif x < -1:
        res = 0
    elif x < 0:
        res = sigma * (np.arcsin(x) + np.pi / 2.0 - np.sqrt(1 - x * x))
    else:
        res = (np.pi - 2.0) * sigma - sigma * (np.arcsin(-x) + np.pi / 2.0 - np.sqrt(1 - x * x))
    return res

def passage_kernel(absolute_difference, sigma):
    # absolute_difference = abs(i-j)
    ans = 0
    if absolute_difference <= sigma:
        ans = 1
    return ans

#Cumulative distribution function of Arc Kernel
def ArcCDF( x,  mean,  sigma):
    res = 0
    x = (x - mean) / sigma
    if x >= 1:
        res = (np.pi - 1.0) * sigma / 2.0
    elif x < -1:
        res = 0
    elif x < 0:
        res = sigma * (np.arcsin(x) + np.pi / 2.0 - np.sqrt(1 - x * x) + (1 - abs(x)) * (1 - abs(x)) / 2.0) / 2.0
    else:
        res = (np.pi - 1.0) * sigma / 2.0 - sigma * (np.arcsin(-x) + np.pi / 2.0 - np.sqrt(1 - x * x) + (1 - abs(x)) * (1 - abs(x)) / 2.0) / 2.0
    return res
            
def cdf(cdf_method, i,  sigma, N):
    return cdf_method(N, i, sigma) - cdf_method(0, i, sigma)
            
def precompute_kernel(kernel_func, sigma):
    max_len = max(document_lengths)+1
    kernel = [0]*max_len
    for i in range(max_len):
        kernel[i] = kernel_func(absolute_difference=i, sigma=sigma)
    return kernel

In [86]:
def plm_retrieval(model_name, kernel_method, kernel_cdf_method, sigma, mu=0):
    """
    mu = 0 means no dirichlet smoothing
    Runs a retrieval method for all the queries and writes the TREC-friendly results in a file.
    
    :param model_name: the name of the model (a string)
    :param score_fn: the scoring function (a function - see below for an example) 
    """
    run_out_path = '{}.run'.format(model_name)

    if os.path.exists(run_out_path):
        return

    retrieval_start_time = time.time()

    print('Retrieving using', model_name)
    data = {}
    
    kernel_at_diff = precompute_kernel(kernel_method, sigma)
    
    count_docs = 0
    count_queries = 0
    for int_doc_id in doc_query_top_n_dic.keys():
        #for each document in the top 1000
        
        count_docs += 1
            
        if count_docs % 1000 == 0:
            print(str(count_docs) + ' docs in: ' +str(time.time() - start_time))
            
        Z = [0]*document_lengths[int_doc_id]
        cdf_params = {'cdf_method':kernel_cdf_method, 'i':0, 'sigma':sigma, 'N':document_lengths[int_doc_id]}
        for i in range(document_lengths[int_doc_id]):
            # Z_i = sum_{w' in V} tf'(w',i;d)
            #for the gaussian kernel Z_i = cummulative_gaussian_kernel(i, sigma, N)
            cdf_params['i'] = i
            Z[i] = cdf(**cdf_params)
        
        for query_id in doc_query_top_n_dic[int_doc_id]:

            query_token_list = tokenized_queries[query_id]
            #for each query of the document in the top 1000    
            ext_doc_id, doc_terms_ids = index.document(int_doc_id)
            
            q_d_score = score_plm_method_document_query(int_doc_id, query_token_list, Z, sigma, mu, kernel_at_diff)
            
            if query_id not in data.keys():
                data[query_id] = []
            data[query_id].append((q_d_score, ext_doc_id))
        
    with open(run_out_path, 'w') as f_out:
        write_run(
            model_name=model_name,
            data=data,
            out_f=f_out,
            max_objects_per_query=1000)

In [111]:
def score_plm_method_document_query(int_doc_id,query_token_list, Z, sigma, mu, kernel_at_index_difference):
    #S(Q,D) = max_{i in [1,N]} {S(Q,D,i)}
    maximum = 0
    for i in range(document_lengths[int_doc_id]):
        #S(Q,D,i) = - sum_{w in query} (p(w|Q)*log(p(w|Q)/p(w|D,i))) 
        score = 0
        for token_id in query_token_list:
            #p(w|D,i))
            doc_plm_at_i = positional_document_language_model(int_doc_id, i, Z[i], sigma, mu, kernel_at_index_difference, token_id)
            #p(w|Q) = count(w in Q) / len(Q)
            word_ml_query_model_estimate = query_token_list.count(token_id) / len(query_token_list)
            
            score += word_ml_query_model_estimate * np.log(word_ml_query_model_estimate / doc_plm_at_i)
        score = score * (-1)
        if score > maximum:
            maximum = score
    return score
                            
def positional_document_language_model(int_doc_id,  i, Z_i, sigma, mu, kernel_at_index_difference, token_id):
    #p(w|d,i) = ( tf'(w,i;d) + mu*p(w|C) ) / ( sum_{w' in V} tf'(w',i;d) + mu )
    if query_term_id not in collection_frequencies.keys():
        tf_collection = 0
    else:
        tf_collection = collection_frequencies[query_term_id]
    
    #computing tf'
    ext_doc_id, doc_token_ids = index.document(int_doc_id)
    indices = [idx for idx, x in enumerate(doc_token_ids) if x == token_id]
    tf_prime = 0
    for idx in indices:
        tf_prime += kernel_at_index_difference[abs(i-idx)]
        
    res = ( tf_prime + mu*tf_collection/total_terms)  / (Z_i + mu)
    return res

In [113]:
pairs = [(gaussian_kernel, GaussianCDF), (cosine_kernel, CosineCDF), (circle_kernel, CircleCDF), (passage_kernel, ArcCDF), (triangle_kernel,TriangleCDF)]
sigma_value = 50
for pair in pairs:
    for mu_value in [100,500,1000,1500]:
        start_time = time.time()
        plm_retrieval('test2_plm_'+pair[0].__name__+':sigma='+str(sigma_value)+':mu='+str(mu_value), pair[0], pair[1], sigma=sigma_value,mu=mu_value)
        print('plm '+pair[0].__name__+' on all query docs took: ' + str(time.time()-start_time))

Retrieving using test2_plm_gaussian_kernel:sigma=50:mu=100
1000 docs in: 122.04963278770447
2000 docs in: 229.45867466926575
3000 docs in: 354.8099286556244
4000 docs in: 473.33420515060425
5000 docs in: 572.0266718864441
6000 docs in: 679.2471206188202
7000 docs in: 784.3018388748169
8000 docs in: 893.2038757801056
9000 docs in: 1003.2750158309937
10000 docs in: 1114.1426339149475
11000 docs in: 1217.6044671535492
plm gaussian_kernel on all query docs took: 1294.124186515808
Retrieving using test2_plm_gaussian_kernel:sigma=50:mu=500
1000 docs in: 130.83012557029724
2000 docs in: 237.73094058036804
3000 docs in: 364.7574737071991
4000 docs in: 485.80920243263245
5000 docs in: 592.1625292301178
6000 docs in: 707.2590334415436
7000 docs in: 820.6258080005646
8000 docs in: 933.1621844768524
9000 docs in: 1052.1746850013733
10000 docs in: 1167.3287465572357
11000 docs in: 1276.0370919704437
plm gaussian_kernel on all query docs took: 1358.0194358825684
Retrieving using test2_plm_gaussian_k

2000 docs in: 214.3794186115265
3000 docs in: 329.7280514240265
4000 docs in: 442.4698996543884
5000 docs in: 537.9499745368958
6000 docs in: 641.5327463150024
7000 docs in: 742.9599940776825
8000 docs in: 849.833687543869
9000 docs in: 960.5624656677246
10000 docs in: 1090.4291679859161
11000 docs in: 1208.5101642608643
plm triangle_kernel on all query docs took: 1290.7984714508057
Retrieving using test2_plm_triangle_kernel:sigma=50:mu=1000
1000 docs in: 123.88587093353271
2000 docs in: 230.79310584068298
3000 docs in: 352.239541053772
4000 docs in: 474.08543610572815
5000 docs in: 575.6894443035126
6000 docs in: 689.5793523788452
7000 docs in: 796.7322874069214
8000 docs in: 908.5812227725983
9000 docs in: 1021.8311941623688
10000 docs in: 1137.7258517742157
11000 docs in: 1243.0424287319183
plm triangle_kernel on all query docs took: 1321.9400098323822
Retrieving using test2_plm_triangle_kernel:sigma=50:mu=1500
1000 docs in: 115.62067461013794
2000 docs in: 212.89122867584229
3000 d

In [116]:
%%bash
echo ' '
./trec_eval/trec_eval -m all_trec -q './ap_88_89/qrel_validation' './tfidf.run' | grep -E "^map\s.*all"
echo ' '
./trec_eval/trec_eval -m all_trec -q './ap_88_89/qrel_validation' './bm25.run' | grep -E "^map\s.*all"
echo ' '
./trec_eval/trec_eval -m all_trec -q './ap_88_89/qrel_validation' './jelinek_mercer:lambda=0.1.run' | grep -E "^map\s.*all"
./trec_eval/trec_eval -m all_trec -q './ap_88_89/qrel_validation' './jelinek_mercer:lambda=0.5.run' | grep -E "^map\s.*all"
./trec_eval/trec_eval -m all_trec -q './ap_88_89/qrel_validation' './jelinek_mercer:lambda=0.9.run' | grep -E "^map\s.*all"
echo ' '
./trec_eval/trec_eval -m all_trec -q './ap_88_89/qrel_validation' './absolute_discounting:delta=0.1.run' | grep -E "^map\s.*all"
./trec_eval/trec_eval -m all_trec -q './ap_88_89/qrel_validation' './absolute_discounting:delta=0.5.run' | grep -E "^map\s.*all"
./trec_eval/trec_eval -m all_trec -q './ap_88_89/qrel_validation' './absolute_discounting:delta=0.9.run' | grep -E "^map\s.*all"
echo ' '
./trec_eval/trec_eval -m all_trec -q './ap_88_89/qrel_validation' './dirichlet_prior:mu=500.run' | grep -E "^map\s.*all"
./trec_eval/trec_eval -m all_trec -q './ap_88_89/qrel_validation' './dirichlet_prior:mu=1000.run' | grep -E "^map\s.*all"
./trec_eval/trec_eval -m all_trec -q './ap_88_89/qrel_validation' './dirichlet_prior:mu=1500.run' | grep -E "^map\s.*all"
echo ' '
./trec_eval/trec_eval -m all_trec -q './ap_88_89/qrel_validation' './plm_gaussian_kernel:sigma=50:mu=0.run' | grep -E "^map\s.*all"
./trec_eval/trec_eval -m all_trec -q './ap_88_89/qrel_validation' './plm_gaussian_kernel:sigma=50:mu=100.run' | grep -E "^map\s.*all"
./trec_eval/trec_eval -m all_trec -q './ap_88_89/qrel_validation' './plm_gaussian_kernel:sigma=50:mu=500.run' | grep -E "^map\s.*all"
./trec_eval/trec_eval -m all_trec -q './ap_88_89/qrel_validation' './plm_gaussian_kernel:sigma=50:mu=1000.run' | grep -E "^map\s.*all"
./trec_eval/trec_eval -m all_trec -q './ap_88_89/qrel_validation' './plm_gaussian_kernel:sigma=50:mu=1500.run' | grep -E "^map\s.*all"
echo ' '
./trec_eval/trec_eval -m all_trec -q './ap_88_89/qrel_validation' './plm_circle_kernel:sigma=50:mu=100.run' | grep -E "^map\s.*all"
./trec_eval/trec_eval -m all_trec -q './ap_88_89/qrel_validation' './plm_circle_kernel:sigma=50:mu=500.run' | grep -E "^map\s.*all"
./trec_eval/trec_eval -m all_trec -q './ap_88_89/qrel_validation' './plm_circle_kernel:sigma=50:mu=1000.run' | grep -E "^map\s.*all"
./trec_eval/trec_eval -m all_trec -q './ap_88_89/qrel_validation' './plm_circle_kernel:sigma=50:mu=1500.run' | grep -E "^map\s.*all"
echo ' '
./trec_eval/trec_eval -m all_trec -q './ap_88_89/qrel_validation' './plm_cosine_kernel:sigma=50:mu=100.run' | grep -E "^map\s.*all"
./trec_eval/trec_eval -m all_trec -q './ap_88_89/qrel_validation' './plm_cosine_kernel:sigma=50:mu=500.run' | grep -E "^map\s.*all"
./trec_eval/trec_eval -m all_trec -q './ap_88_89/qrel_validation' './plm_cosine_kernel:sigma=50:mu=1000.run' | grep -E "^map\s.*all"
./trec_eval/trec_eval -m all_trec -q './ap_88_89/qrel_validation' './plm_cosine_kernel:sigma=50:mu=1500.run' | grep -E "^map\s.*all"
echo ' '
./trec_eval/trec_eval -m all_trec -q './ap_88_89/qrel_validation' './plm_passage_kernel:sigma=50:mu=100.run' | grep -E "^map\s.*all"
./trec_eval/trec_eval -m all_trec -q './ap_88_89/qrel_validation' './plm_passage_kernel:sigma=50:mu=500.run' | grep -E "^map\s.*all"
./trec_eval/trec_eval -m all_trec -q './ap_88_89/qrel_validation' './plm_passage_kernel:sigma=50:mu=1000.run' | grep -E "^map\s.*all"
./trec_eval/trec_eval -m all_trec -q './ap_88_89/qrel_validation' './plm_passage_kernel:sigma=50:mu=1500.run' | grep -E "^map\s.*all"
echo ' '
./trec_eval/trec_eval -m all_trec -q './ap_88_89/qrel_validation' './plm_triangle_kernel:sigma=50:mu=100.run' | grep -E "^map\s.*all"
./trec_eval/trec_eval -m all_trec -q './ap_88_89/qrel_validation' './plm_triangle_kernel:sigma=50:mu=500.run' | grep -E "^map\s.*all"
./trec_eval/trec_eval -m all_trec -q './ap_88_89/qrel_validation' './plm_triangle_kernel:sigma=50:mu=1000.run' | grep -E "^map\s.*all"
./trec_eval/trec_eval -m all_trec -q './ap_88_89/qrel_validation' './plm_triangle_kernel:sigma=50:mu=1500.run' | grep -E "^map\s.*all"

 
map                   	all	0.2403
 
map                   	all	0.2196
 
map                   	all	0.1366
map                   	all	0.1366
map                   	all	0.1366
 
map                   	all	0.1367
map                   	all	0.1404
map                   	all	0.1397
 
map                   	all	0.1758
map                   	all	0.1733
map                   	all	0.1725
 
map                   	all	0.0998
map                   	all	0.1534
map                   	all	0.1528
map                   	all	0.1525
map                   	all	0.1523
 
map                   	all	0.1245
map                   	all	0.1245
map                   	all	0.1245
map                   	all	0.1247
 
map                   	all	0.1218
map                   	all	0.1216
map                   	all	0.1216
map                   	all	0.1214
 
map                   	all	0.1220
map                   	all	0.1220
map                   	all	0.1220
map                   	all	0.1220
 
map                   	all	0

In [119]:
%%bash
./trec_eval/trec_eval -m all_trec -q './ap_88_89/qrel_test' './plm_gaussian_kernel:sigma=50:mu=100.run' | grep -E "^map\s.*all"

map                   	all	0.1354


In [43]:
%%bash
a1=($(./trec_eval/trec_eval -m all_trec -q './ap_88_89/qrel_validation' './plm_circle_kernel:sigma=50:mu=100.run' | grep -E "_100 "))
a2=($(./trec_eval/trec_eval -m all_trec -q './ap_88_89/qrel_validation' './plm_circle_kernel:sigma=50:mu=500.run' | grep -E "_100 "))
a3=($(./trec_eval/trec_eval -m all_trec -q './ap_88_89/qrel_validation' './plm_circle_kernel:sigma=50:mu=1000.run' | grep -E "_100 "))
a4=($(./trec_eval/trec_eval -m all_trec -q './ap_88_89/qrel_validation' './plm_circle_kernel:sigma=50:mu=1500.run' | grep -E "_100 "))
for index in "${!a1[@]}"
do
    echo "$index ${a1[index]} ${a2[index]} ${a3[index]} ${a4[index]}"
done

0 P_100 P_100 P_100 P_100
1 103 103 103 103
2 0.3000 0.3000 0.3000 0.3000
3 recall_100 recall_100 recall_100 recall_100
4 103 103 103 103
5 0.5455 0.5455 0.5455 0.5455
6 ndcg_cut_100 ndcg_cut_100 ndcg_cut_100 ndcg_cut_100
7 103 103 103 103
8 0.4171 0.4171 0.4171 0.4171
9 map_cut_100 map_cut_100 map_cut_100 map_cut_100
10 103 103 103 103
11 0.1510 0.1510 0.1510 0.1510
12 relative_P_100 relative_P_100 relative_P_100 relative_P_100
13 103 103 103 103
14 0.5455 0.5455 0.5455 0.5455
15 P_100 P_100 P_100 P_100
16 111 111 111 111
17 0.7000 0.7000 0.7000 0.7000
18 recall_100 recall_100 recall_100 recall_100
19 111 111 111 111
20 0.6250 0.6250 0.6250 0.6250
21 ndcg_cut_100 ndcg_cut_100 ndcg_cut_100 ndcg_cut_100
22 111 111 111 111
23 0.7161 0.7161 0.7161 0.7161
24 map_cut_100 map_cut_100 map_cut_100 map_cut_100
25 111 111 111 111
26 0.4611 0.4611 0.4611 0.4611
27 relative_P_100 relative_P_100 relative_P_100 relative_P_100
28 111 111 111 111
29 0.7000 0.7000 0.7000 0.7000
30 P_100 P_100 P_100 P_1

In [43]:
import subprocess
#Getting evaluation measures
def parse_out(out):
    return None
def eval(run_doc):
    # run_doc should be a filename
    qrel_doc = "/ap_88_89/qrel_test"
    qrel_doc = "example.qrel"
    grep_string = "^ndcg_cut_10\s|^map_cut_1000\s|^P_5\s|^recall_1000\s"
    eval_command = ["./trec_eval/trec_eval","-m","all_trec","-q",qrel_doc,run_doc]
    grep_command = ["grep","-E",grep_string]
    p = subprocess.Popen(eval_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    p_grep = subprocess.Popen(grep_command,stdin=p.stdout,stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out,err = p_grep.communicate()
    print(out.decode("utf-8"))
    print(err.decode("utf-8"))

In [44]:
eval("example.run")

P_5                   	Q1	0.2000
recall_1000           	Q1	1.0000
ndcg_cut_10           	Q1	1.0000
map_cut_1000          	Q1	1.0000
P_5                   	Q2	0.2000
recall_1000           	Q2	1.0000
ndcg_cut_10           	Q2	0.6309
map_cut_1000          	Q2	0.5000
P_5                   	all	0.2000
recall_1000           	all	1.0000
ndcg_cut_10           	all	0.8155
map_cut_1000          	all	0.7500




### Task 2: Latent Semantic Models (LSMs) [15 points] ###

In this task you will experiment with applying distributional semantics methods ([LSI](http://lsa3.colorado.edu/papers/JASIS.lsi.90.pdf) **[5 points]** and [LDA](https://www.cs.princeton.edu/~blei/papers/BleiNgJordan2003.pdf) **[5 points]**) for retrieval.

You do not need to implement LSI or LDA on your own. Instead, you can use [gensim](http://radimrehurek.com/gensim/index.html). An example on how to integrate Pyndri with Gensim for word2vec can be found [here](https://github.com/cvangysel/pyndri/blob/master/examples/word2vec.py). For the remaining latent vector space models, you will need to implement connector classes (such as `IndriSentences`) by yourself.

In order to use a latent semantic model for retrieval, you need to:
   * build a representation of the query **q**,
   * build a representation of the document **d**,
   * calculate the similarity between **q** and **d** (e.g., cosine similarity, KL-divergence).
     
The exact implementation here depends on the latent semantic model you are using. 
   
Each of these LSMs come with various hyperparameters to tune. Make a choice on the parameters, and explicitly mention the reasons that led you to these decisions. You can use the validation set to optimize hyper parameters you see fit; motivate your decisions. In addition, mention clearly how the query/document representations were constructed for each LSM and explain your choices.

In this experiment, you will first obtain an initial top-1000 ranking for each query using TF-IDF in **Task 1**, and then re-rank the documents using the LSMs. Use TREC Eval to obtain the results and report on `NDCG@10`, Mean Average Precision (`MAP@1000`), `Precision@5` and `Recall@1000`.

Perform significance testing **[5 points]** (similar as in Task 1) in the class of semantic matching methods.

### Task 3:  Word embeddings for ranking [20 points] (open-ended) ###

First create word embeddings on the corpus we provided using [word2vec](http://arxiv.org/abs/1411.2738) -- [gensim implementation](https://radimrehurek.com/gensim/models/word2vec.html). You should extract the indexed documents using pyndri and provide them to gensim for training a model (see example [here](https://github.com/nickvosk/pyndri/blob/master/examples/word2vec.py)).
   
This is an open-ended task. It is left up you to decide how you will combine word embeddings to derive query and document representations. Note that since we provide the implementation for training word2vec, you will be graded based on your creativity on combining word embeddings for building query and document representations.

Note: If you want to experiment with pre-trained word embeddings on a different corpus, you can use the word embeddings we provide alongside the assignment (./data/reduced_vectors_google.txt.tar.gz). These are the [google word2vec word embeddings](https://code.google.com/archive/p/word2vec/), reduced to only the words that appear in the document collection we use in this assignment.

### Task 4: Learning to rank (LTR) [15 points] (open-ended) ###

In this task you will get an introduction into learning to rank for information retrieval.

You can explore different ways for devising features for the model. Obviously, you can use the retrieval methods you implemented in Task 1, Task 2 and Task 3 as features. Think about other features you can use (e.g. query/document length). Creativity on devising new features and providing motivation for them will be taken into account when grading.

For every query, first create a document candidate set using the top-1000 documents using TF-IDF, and subsequently compute features given a query and a document. Note that the feature values of different retrieval methods are likely to be distributed differently.

You are adviced to start some pointwise learning to rank algorithm e.g. logistic regression, implemented in [scikit-learn](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html).
Train your LTR model using 10-fold cross validation on the test set. More advanced learning to rank algorithms will be appreciated when grading.

### Task 4: Write a report [15 points; instant FAIL if not provided] ###

The report should be a PDF file created using the [sigconf ACM template](https://www.acm.org/publications/proceedings-template) and will determine a significant part of your grade.

   * It should explain what you have implemented, motivate your experiments and detail what you expect to learn from them. **[10 points]**
   * Lastly, provide a convincing analysis of your results and conclude the report accordingly. **[10 points]**
      * Do all methods perform similarly on all queries? Why?
      * Is there a single retrieval model that outperforms all other retrieval models (i.e., silver bullet)?
      * ...

**Hand in the report and your self-contained implementation source files.** Only send us the files that matter, organized in a well-documented zip/tgz file with clear instructions on how to reproduce your results. That is, we want to be able to regenerate all your results with minimal effort. You can assume that the index and ground-truth information is present in the same file structure as the one we have provided.


In [1]:
import gensim
import os
from pprint import pprint

model_folder = 'models/task3/'
model_name = 'init-word-embbedings'
use_pretrained = True

if use_pretrained:
    model = gensim.models.KeyedVectors.load_word2vec_format('reduced_vectors_google.txt')
else:
    model = gensim.models.Word2Vec.load(model_folder + model_name + '_v' + str(5))

In [2]:
import math
import numpy

def calculate_centroid(int_document_id):
    _, doc_terms_ids = index.document(int_document_id)
    
    terms = [id2token[word_id] for word_id in doc_terms_ids if word_id > 0]
    doc_length = len(doc_terms_ids)
    sum_of_normalized_vectors = 0
    skipped_terms = 0
    
    if doc_length == 0:
        print('Doc length 0, why? doc_id: ' + str(int_document_id))
        return False
    
    for term in terms:
        if term in model.wv:
            word2vec_repr = model.wv[term]
            l2_norm = math.sqrt(sum([i*i for i in word2vec_repr]))
            sum_of_normalized_vectors += word2vec_repr / l2_norm
        else:
            skipped_terms += 1
    
    if skipped_terms == len(terms):
        print('All terms skipped, why? doc_id: ' + str(int_document_id))
        return False
    
    return sum_of_normalized_vectors / doc_length

In [3]:
def func():
    
    for int_doc_id in range(index.document_base(),index.maximum_document()):
        print(calculate_centroid(int_doc_id).shape)
        break

In [8]:
func()

(300,)


In [9]:
"""
Implementation of pairwise ranking using scikit-learn LinearSVC
Reference: 
    "Large Margin Rank Boundaries for Ordinal Regression", R. Herbrich,
    T. Graepel, K. Obermayer 1999
    "Learning to rank from medical imaging data." Pedregosa, Fabian, et al., 
    Machine Learning in Medical Imaging 2012.
Authors: Fabian Pedregosa <fabian@fseoane.net>
         Alexandre Gramfort <alexandre.gramfort@inria.fr>
See also https://github.com/fabianp/pysofia for a more efficient implementation
of RankSVM using stochastic gradient descent methdos.
"""

import itertools
import numpy as np

from sklearn import svm, linear_model, cross_validation


def transform_pairwise(X, y):
    """Transforms data into pairs with balanced labels for ranking
    Transforms a n-class ranking problem into a two-class classification
    problem. Subclasses implementing particular strategies for choosing
    pairs should override this method.
    In this method, all pairs are choosen, except for those that have the
    same target value. The output is an array of balanced classes, i.e.
    there are the same number of -1 as +1
    Parameters
    ----------
    X : array, shape (n_samples, n_features)
        The data
    y : array, shape (n_samples,) or (n_samples, 2)
        Target labels. If it's a 2D array, the second column represents
        the grouping of samples, i.e., samples with different groups will
        not be considered.
    Returns
    -------
    X_trans : array, shape (k, n_feaures)
        Data as pairs
    y_trans : array, shape (k,)
        Output class labels, where classes have values {-1, +1}
    """
    X_new = []
    y_new = []
    y = np.asarray(y)
    if y.ndim == 1:
        y = np.c_[y, np.ones(y.shape[0])]
    comb = itertools.combinations(range(X.shape[0]), 2)
    for k, (i, j) in enumerate(comb):
        if y[i, 0] == y[j, 0] or y[i, 1] != y[j, 1]:
            # skip if same target or different group
            continue
        X_new.append(X[i] - X[j])
        y_new.append(np.sign(y[i, 0] - y[j, 0]))
        # output balanced classes
        if y_new[-1] != (-1) ** k:
            y_new[-1] = - y_new[-1]
            X_new[-1] = - X_new[-1]
    return np.asarray(X_new), np.asarray(y_new).ravel()


class RankSVM(svm.LinearSVC):
    """Performs pairwise ranking with an underlying LinearSVC model
    Input should be a n-class ranking problem, this object will convert it
    into a two-class classification problem, a setting known as
    `pairwise ranking`.
    See object :ref:`svm.LinearSVC` for a full description of parameters.
    """

    def fit(self, X, y):
        """
        Fit a pairwise ranking model.
        Parameters
        ----------
        X : array, shape (n_samples, n_features)
        y : array, shape (n_samples,) or (n_samples, 2)
        Returns
        -------
        self
        """
        X_trans, y_trans = transform_pairwise(X, y)
        super(RankSVM, self).fit(X_trans, y_trans)
        return self

    def decision_function(self, X):
        return np.dot(X, self.coef_.ravel())

    def predict(self, X):
        """
        Predict an ordering on X. For a list of n samples, this method
        returns a list from 0 to n-1 with the relative order of the rows of X.
        The item is given such that items ranked on top have are
        predicted a higher ordering (i.e. 0 means is the last item
        and n_samples would be the item ranked on top).
        Parameters
        ----------
        X : array, shape (n_samples, n_features)
        Returns
        -------
        ord : array, shape (n_samples,)
            Returns a list of integers representing the relative order of
            the rows in X.
        """
        if hasattr(self, 'coef_'):
            return np.argsort(np.dot(X, self.coef_.ravel()))
        else:
            raise ValueError("Must call fit() prior to predict()")

    def score(self, X, y):
        """
        Because we transformed into a pairwise problem, chance level is at 0.5
        """
        X_trans, y_trans = transform_pairwise(X, y)
        return np.mean(super(RankSVM, self).predict(X_trans) == y_trans)

/home/nichita/pytorch/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [20]:
def get_features(doc_id, query):
    # Other features?
    # - some kind of cosine distance between word2vec for doc and query
    # - LDA outputs
    f_doc_len = np.log(document_lengths[doc_id])
    f_query_len = np.log(len(query))
    f_coverage_score = coverage_score(doc_id, query)
    
    f_tfidf = get_tfidf_score(doc_id, query)
    f_bm25 = BM25_factory()(doc_id, query)
    f_jm = jm_factory(best_param["jm"])(doc_id, query)
    f_abs_disc = abs_disc_factory(best_param["abs_disc"])(doc_id, query)
    return np.asarray([f_doc_len, f_query_len, f_coverage_score, f_tfidf, f_bm25, f_jm, f_abs_disc])

    
def create_dataset(qrel_file):
    with open('ap_88_89/topics_title', 'r') as fd:
        queries = parse_topics([fd])
    
    x = []
    y = []
    #68 0 AP880218-0195 1
    with open(qrel_file, 'r') as fd:
        for line in fd:
            q_id,  _, ext_doc_id, true_score = line.split(" ")
            
            if q_id in queries:
                query_tokens = index.tokenize(queries[q_id])
                query = [token2id.get(query_token,0) for query_token in query_tokens]
                query = [word_id for word_id in query if word_id > 0]
                
                if not index.document_ids([ext_doc_id]):
                    continue
                    
                int_doc_id = index.document_ids([ext_doc_id])[0][1]
                doc = index.document(int_doc_id)

                x.append(get_features(doc_id, query))
                y.append(float(true_score[0]))
        
    return (np.asarray(x), np.asarray(y))
    
create_dataset('ap_88_89/qrel_test')

NameError: name 'make_features' is not defined

In [ ]:
if __name__ == '__main__':
    # as showcase, we will create some non-linear data
    # and print the performance of ranking vs linear regression

    np.random.seed(1)
    n_samples, n_features = 300, 5
    true_coef = np.random.randn(n_features)
    X = np.random.randn(n_samples, n_features)
    noise = np.random.randn(n_samples) / np.linalg.norm(true_coef)
    y = np.dot(X, true_coef)
    y = np.arctan(y) # add non-linearities
    y += .1 * noise  # add noise
    Y = np.c_[y, np.mod(np.arange(n_samples), 5)]  # add query fake id
    cv = cross_validation.KFold(n_samples, 5)
    train, test = iter(cv).next()

    # make a simple plot out of it
    import pylab as pl
    pl.scatter(np.dot(X, true_coef), y)
    pl.title('Data to be learned')
    pl.xlabel('<X, coef>')
    pl.ylabel('y')
    pl.show()

    # print the performance of ranking
    rank_svm = RankSVM().fit(X[train], Y[train])
    print 'Performance of ranking ', rank_svm.score(X[test], Y[test])

    # and that of linear regression
    ridge = linear_model.RidgeCV(fit_intercept=True)
    ridge.fit(X[train], y[train])
    X_test_trans, y_test_trans = transform_pairwise(X[test], y[test])
    score = np.mean(np.sign(np.dot(X_test_trans, ridge.coef_)) == y_test_trans)
print 'Performance of linear regression ', score